A World map - most popular cartogrpahic methods

In [ ]:
import os
os.chdir(r'd:\git20\GeoPython_Ukraine2024')

import warnings 
warnings.filterwarnings('ignore')

In [1]:
# imports 

#### resources:
- https://geopandas.org/en/v0.9.0/docs/user_guide/mapping.html
- https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population

In [ ]:
# countries geodatable r'geoBoundariesCGAZ_ADM0.shp'

In [25]:
# join tables df1.merge(df2, left_on='lkey', right_on='rkey') => gdf_cntrs_pop

# reproject epsg_out = 'ESRI:54030'

In [ ]:
# plot a variable on a map

In [ ]:
# Calculate relative value

In [ ]:
# plot relative variable

In [ ]:
# graduated colors - different ways of dividing into classes
# schemes = ['boxplot', 'equalinterval', 'fisherjenks',  'fisherjenkssampled','headtailbreaks']

In [ ]:
# Graduated symbols visualisation - all centroids - are they ok? => gdf_cntrs_centr

In [ ]:
# France centroid example

In [54]:
# fix problems with the multipolygons and multicentroids; explode, sort by area and drop duplicates

In [ ]:
# France example again

In [ ]:
# plot graduated symbols

In [ ]:
# plot semi-final map: auto assing classes and colors, graduated color and symbols

In [ ]:
# better classification
# levels = [0, 10, 25, 50, 100, 200, 1500]
# colors = ['#ffffd4', '#fee391', '#fec44f', '#fe9929', '#d95f0e', '#993404']

In [16]:
# show that anything - a shape or a text may be plotted on a map - and what about it?
# ax.scatter(x,y,size,marker,color)

Plot a great world base map

In [63]:
epsg_out = 'ESRI:54030'

from pyproj import Proj, transform
import matplotlib.patches as patches
outProj = Proj(epsg_out)  
inProj = Proj('epsg:4326') 

def plot_labels(ax):
    for y, x, name in zip([80, -145, -130, -130], 
                        [68, 25, -22.5, -65],  
                        ['North Arctic Circle', 'Tropic of Cancer', 
                        'Tropic od Capicorn', 'South Arctic Circle']):

        x2,y2 = transform(inProj,outProj, x, y)
        ax.text(x2, y2, name, color='gray', fontsize=6)


    for lon in [0, 30, 60, 90, 120, 150]:
        for sign in [-1,1]:
            for sign_2 in [-1,1]:
                x2, y2 = transform(inProj,outProj, 90 * sign_2, lon * sign)
                if sign_2 > 0:
                    y2 = y2 + 100_000
                else:
                    y2 = y2 - 350_000
                x2 = x2 - 320_000
                ax.text(x2, y2, str(lon) + '°', color='gray', fontsize=7)


    def zaokragl_do_wielokrotnosci_10(liczba):
        return abs(round(liczba / 10) * 10)

    for lat, x_offset in zip([-61, -31, 0, 30, 60], [300_000, 100_000, 20_000, 10_000, 10_000]):
        x2, y2 = transform(inProj, outProj, lat, 180)
        ax.text(x2 + x_offset, y2, str(zaokragl_do_wielokrotnosci_10(lat)) + '°', color='gray', fontsize=7)

    for lat, x_offset in zip([-61, -31, 0, 30, 60], [700_000, 500_000, 300_000, 450_000, 400_000]):
        x2, y2 = transform(inProj, outProj, lat, -180)
        ax.text(x2 - x_offset, y2, str(zaokragl_do_wielokrotnosci_10(lat)) + '°', color='gray', fontsize=7)

    return ax


def plot_base_map():
    # Extra layers
    # Graticules
    graticules = gpd.read_file(r'.\data\graticules_10.shp')
    graticules_select = graticules[graticules.degrees.isin([0,30,60,90,120,150])]
    graticules_select = graticules_select.to_crs(epsg_out)

    # Oceans
    gdf_ocean = gpd.read_file(r'.\data\oceans.shp')
    gdf_ocean = gdf_ocean.to_crs(epsg_out)

    # tropics
    geo_lines = gpd.read_file(r'.\data\geo_lines.shp')
    geo_lines = geo_lines.to_crs(epsg_out)
    geo_lines = geo_lines[geo_lines.featurecla =='Circle of latitude']

    # world
    world = gpd.read_file(r'.\data\world.shp')
    world = world.to_crs(epsg_out)

    fig, ax = plt.subplots(figsize=(18,10))
    gdf_ocean.plot(ax=ax, color='#bee8ff')
    world.buffer(5_000).boundary.plot(ax=ax, color='black', linewidth=0.5)
    world.buffer(600_000).boundary.plot(ax=ax, color='black', linewidth=1.2)
    graticules_select.plot(ax=ax, color='gray', linewidth=0.4)
    geo_lines.plot(ax=ax, color='gray', linewidth=0.4, linestyle='--')
    ax.axis('off')
    return ax


def plot_main_map(ax, colors):
    levels = [0, 10, 25, 50, 100, 200, 1500]
    cmap, norm = matplotlib.colors.from_levels_and_colors(levels, colors)

    gdf_cntrs_pop.plot(column='pop_relative', cmap=cmap, norm=norm, ax=ax)
    gdf_cntrs_pop.boundary.plot(linewidth=0.2, color='gray', ax=ax)
    gdf_cntrs_centr.plot(markersize=gdf_cntrs_centr['Population']/2_000_000,
                                    alpha=0.7, color='gray', ax=ax)
    return ax



def plot_legend(ax, colors):
    x_loc = -10_000_000
    offset = 750_000
      
    for color, name in zip(colors, ['<10', '10-25', '25-50', '50-100', '100-200', '>200']):
        x_loc = x_loc + offset
        rect = patches.Rectangle((-19_500_000, x_loc), 700_000, 500_000, facecolor=color)
        ax.text(-18_500_000, x_loc + 120_000, name, size=10)
        ax.add_patch(rect)
    
    ax.text(-19_500_000, -4_700_000, 'people per sq. km', fontsize=7)

    # circles 
    circle = ax.scatter(-15_800_000, x_loc - 3_200_000, s=1000, marker='o', color='gray', alpha=0.3)
    circle = ax.scatter(-15_800_000, x_loc - 3_500_000, s=250, marker='o', color='gray', alpha=0.3)

    ax.text(-14_800_000, x_loc - 2_800_000, '1000M', fontsize=10)
    ax.text(-14_800_000,  x_loc - 3_500_000, '250M', fontsize=10)
    return ax


def plot_title(ax):
    ax.text(-19_500_000, 9_000_000, 'The World', fontsize=20)
    ax.text(-19_500_000, 8_000_000, 'Population', fontsize=14)
    return ax


In [ ]:
# subsequently call functions
ax = plot_base_map()
ax = plot_labels(ax)
colors = ['#ffffd4', '#fee391', '#fec44f', '#fe9929', '#d95f0e', '#993404']
ax = plot_main_map(ax, colors)
ax = plot_legend(ax, colors)
ax = plot_title(ax)